In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import astropy.coordinates as coord

In [2]:
path = '/ocean/projects/phy210068p/hsu1/Ananke_datasets_training/GaiaDR3_data_reduced.hdf5'
with h5py.File(path, 'r') as f:
    print(list(f.keys()))
    is_accreted = f['is_accreted'][:]
    feh = f['feh'][:]
    ra = f['ra'][:]

print(is_accreted.shape),   print(feh.shape),   print(ra.shape)

['dec', 'feh', 'is_accreted', 'parallax', 'pmdec', 'pmra', 'ra', 'radial_velocity', 'source_id']
(29684295,)
(29684295,)
(29684295,)


(None, None, None)

In [ ]:
with h5py.File(path, 'r') as f:
    ra = f['ra'][:]
    dec = f['dec'][:]
    parallax = f['parallax'][:]
    pmra = f['pmra'][:]
    pmdec = f['pmdec'][:]
    rv = f['radial_velocity'][:]
    source_id = f['source_id'][:]
    keep_source_id = source_id
    feh = f['feh'][:]

In [ ]:
non_null_mask = (~np.isnan(ra))
source_id = source_id[non_null_mask]
ra = ra[non_null_mask]
dec = dec[non_null_mask]
parallax = parallax[non_null_mask]
pmra = pmra[non_null_mask]
pmdec = pmdec[non_null_mask]
rv = rv[non_null_mask]
feh = feh[non_null_mask]

In [ ]:
ra = ra * u.deg
dec = dec * u.deg
pmra = pmra * u.mas / u.yr
pmdec = pmdec * u.mas / u.yr
parallax = parallax * u.mas
rv = rv * u.km / u.s

dist = coord.Distance(parallax=parallax, allow_negative=True)

# Coord transformation
icrs = coord.ICRS(
    ra=ra, dec=dec, distance=dist, pm_ra_cosdec=pmra, pm_dec=pmdec, radial_velocity=rv)
gal = icrs.transform_to(coord.Galactocentric())
x_gal = gal.x.to_value(u.kpc)
y_gal = gal.y.to_value(u.kpc)
z_gal = gal.z.to_value(u.kpc)
vx_gal = gal.v_x.to_value(u.km/u.s)
vy_gal = gal.v_y.to_value(u.km/u.s)
vz_gal = gal.v_z.to_value(u.km/u.s)  

In [ ]:
z_mask = (np.absolute(z_gal) > 1.5)
source_id_z = source_id[z_mask]
vx_z = vx_gal[z_mask]
vy_z = vy_gal[z_mask]
vz_z = vz_gal[z_mask]
feh_z = feh[z_mask]

In [ ]:
fe_mask = (feh_z < -1.5)
source_id_z_fe = source_id_z[fe_mask]
vx_z_fe = vx_z[fe_mask]
vy_z_fe = vy_z[fe_mask]
vz_z_fe = vz_z[fe_mask]
feh_z_fe = feh_z[fe_mask]

In [ ]:
accretion_id = np.isin(keep_source_id, source_id_z_fe)
accretion = np.int32(accretion_id)
is_accreted = accretion
with h5py.File(path, 'a') as f:
    # del f['is_accreted']
    f.create_dataset('is_accreted', data=accretion)

In [ ]:
accretion.shape, keep_source_id.shape, np.sum(accretion) / len(keep_source_id)

In [ ]:
vr_z_fe = np.sqrt(vx_z_fe**2 + vz_z_fe**2)

In [ ]:
np.sum(np.isnan(x_gal))

In [ ]:
np.sum(np.isnan(vx_z))

In [ ]:
fig, ax = plt.subplots(1, figsize=(13, 10))

h = ax.hist2d(vy_z_fe, vr_z_fe, bins=100, weights=np.repeat(1/len(vx_z_fe), len(vx_z_fe)), norm=mpl.colors.LogNorm())
ax.set_xlabel(r'$V_y [km s^{-1}]$', fontsize=35)
ax.set_ylabel(r'$\sqrt{V_x^2+V_z^2} [km s^{-1}]$', fontsize=35)
ax.set_title('Gaia DR3 Accreted Stars (ZM Cut)', fontsize=40)
cb = fig.colorbar(h[3], ax=ax, label=r'Normalized Counts (km/s)$^{-2}$') 
cb.ax.tick_params(labelsize=35)
ax.tick_params(axis='both', labelsize=35)

In [ ]:
with h5py.File(path, 'r') as f:
    print(list(f.keys()))